In [12]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd

print("hola")

import spacy
from spacy.util import minibatch, compounding

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


nlp = spacy.blank('en')

textcat = nlp.create_pipe('textcat')
nlp.add_pipe(textcat, last=True)


PrimaryEmotion = pd.read_csv('emotion.xls.csv')
ProjectData = PrimaryEmotion[['emotion','sentence']]


emotion_list = ProjectData['emotion']
sentence_list = ProjectData['sentence']

print("hola")


sorted_list = []
sorted_emo = []
c = [emotion_list, sentence_list]


#######Sorts and counts Happy emotions
for i in range(len(c[1])):
    sorted_list.append(sentence_list[i])
    if (c[0][i] == 'Joy' or c[0][i] == 'Love' or c[0][i] == 'Optimism' or c[0][i] == 'Awe' or c[0][i] == 'Trust'):
        sorted_emo.append("happy")
    else:
        sorted_emo.append("sad")



max_fatures = 8000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(sentence_list)
X = tokenizer.texts_to_sequences(sentence_list)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(18,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Y = pd.get_dummies(emotion_list).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 128
model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size, verbose = 2)

validation_size = 50



hola
hola
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 75, 128)           1024000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 75, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 18)                3546      
Total params: 1,282,346
Trainable params: 1,282,346
Non-trainable params: 0
_________________________________________________________________
None
(1691, 75) (1691, 18)
(833, 75) (833, 18)
Epoch 1/50
 - 10s - loss: 2.7761 - acc: 0.1904
Epoch 2/50
 - 8s - loss: 2.5941 - acc: 0.2052
Epoch 3/50
 - 7s - loss: 2.5676 - acc: 0.2123
Epoch 4/50
 - 7s - loss: 2.5441 - acc: 0.2123
Epoch 5/50
 - 7s - los

In [14]:
validation_size = 200

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))




score: 5.01
acc: 0.18
